In [14]:
import pandas as pd
features = ['accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'number_of_reviews']
dc_listings = pd.read_csv('listings.csv')
print(dc_listings.shape)
dc_listings.head()

(3723, 92)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11
2,8249488,https://www.airbnb.com/rooms/8249488,20151002231825,2015-10-03,Spacious/private room for single,This is an ideal room for a single traveler th...,NaN,This is an ideal room for a single traveler th...,none,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,1.00
3,8409022,https://www.airbnb.com/rooms/8409022,20151002231825,2015-10-03,A wonderful bedroom with library,Prime location right on the Potomac River in W...,NaN,Prime location right on the Potomac River in W...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,1,NaN
4,8411173,https://www.airbnb.com/rooms/8411173,20151002231825,2015-10-03,Downtown Silver Spring,"Hi travellers! I live in this peaceful spot, b...",This is a 750 sq ft 1 bedroom 1 bathroom. Whi...,"Hi travellers! I live in this peaceful spot, b...",none,Silver Spring is booming. You can walk to a n...,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


In [15]:
import numpy as np
our_acc_value = 3# 有三个房间
dc_listings['distance'] = np.abs(dc_listings['accommodates']-our_acc_value)# 房间数与3的差值
dc_listings['distance'].value_counts().sort_index()# 统计各差值数量

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [16]:
dc_listings = dc_listings.sample(frac=1, random_state=0)# 将数据打乱顺序
dc_listings = dc_listings.sort_values('distance')# 根据距离排序
dc_listings[['distance', 'price']].head()

,distance,price
2645,0,$75.00
2825,0,$120.00
2145,0,$90.00
2541,0,$50.00
3349,0,$105.00


In [17]:
dc_listings['price'] = dc_listings['price'].str.replace('\$|,','').astype(float)# 将$符号替换掉
mean_price = dc_listings['price'].iloc[:5].mean()# 计算价格平均值
mean_price

88.0

In [18]:
dc_listings.drop('distance', axis=1, inplace=True)

train_df = dc_listings.copy().iloc[:2792]# 训练集
test_df = dc_listings.copy().iloc[2792:]# 测试集

In [19]:
# train_df.head()

In [20]:
# 基于单变量预测价格
def predict_price(new_listing_value, feature_column):
    temp_df = train_df
#     temp_df['distance'] = np.abs(dc_listings[feature_column]-new_listing_value)# 计算距离
    temp_df['distance'] = np.abs(train_df[feature_column]-new_listing_value)# 计算距离
    temp_df = temp_df.sort_values('distance')# 按距离排序
    knn_5 = temp_df['price'].iloc[:5]# 取前5条数据，k=5
    predict_price = knn_5.mean()#取平均值进行预测
    return predict_price

In [21]:
test_df['predict_price'] = test_df['accommodates'].apply(predict_price, feature_column='accommodates')# apply，对每条样本执行同样的操作

In [22]:
# 计算均方根误差RMSE
test_df['squared_error'] = (test_df['predict_price']-test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse# 对于一个变量的模型评估得分，误差过大

212.98927967051529

In [23]:
for feature in ['accommodates', 'bedrooms', 'bathrooms', 'beds', 'number_of_reviews']:
    test_df['predicted_price'] = test_df[feature].apply(predict_price, feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price']-test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print('RMSE for the {} coloumn: {}'.format(feature, rmse))

RMSE for the accommodates coloumn: 212.9892796705153
RMSE for the bedrooms coloumn: 199.80935328065047
RMSE for the bathrooms coloumn: 230.2471670568422
RMSE for the beds coloumn: 227.7858424753905
RMSE for the number_of_reviews coloumn: 235.91327066995507


In [32]:
# 利用sklearn库进行特征标准化进行数据预处理
from sklearn.preprocessing import StandardScaler
features = ['accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'number_of_reviews']
dc_listings = pd.read_csv('listings.csv')
dc_listings = dc_listings[features]
dc_listings['price'] = dc_listings['price'].str.replace('\$|,','').astype(float)# 处理$
dc_listings = dc_listings.dropna()# 去掉缺失值
dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])# 特征标准化处理
normalized_listings = dc_listings
print(dc_listings.shape)
normalized_listings.head()

d:\python\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
d:\python\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(3671, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.401420,-0.249501,-0.439211,0.297386,0.081119,-0.341421,-0.016575,-0.516779
1,1.399466,2.129508,2.969551,1.141704,1.462622,-0.065047,-0.016606,1.706767
2,-1.095648,-0.249501,1.265170,-0.546933,-0.718699,-0.065047,-0.016575,-0.482571
3,-0.596625,-0.249501,-0.439211,-0.546933,-0.391501,-0.341421,-0.016575,-0.516779
4,0.401420,-0.249501,-0.439211,-0.546933,-0.718699,1.316824,-0.016575,-0.516779


In [33]:
norm_train_df = normalized_listings.copy().iloc[:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

In [35]:
from scipy.spatial import distance# 使用scipy库计算欧式距离

first_listings = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listings = normalized_listings.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listings, fifth_listings)
first_fifth_distance

3.723019604017032

In [37]:
# 多变量KNN模型
def predict_price_multivariate(new_listing_value, feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns], [new_listing_value[feature_columns]])# 求欧式距离
    temp_df = temp_df.sort_values('distance')# 根据距离排序
    knn5 = temp_df['price'].iloc[:5]# 取前5个值
    predicted_price = knn5.mean()# 求平均数
    return predicted_price

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate, feature_columns=cols, axis=1)
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price']) ** (2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

0.789406392258


In [42]:
# 使用Sklearn完成KNN
from sklearn.neighbors import KNeighborsRegressor
cols = ['accommodates', 'bathrooms']
knn = KNeighborsRegressor(n_neighbors=10)# 设置knn，默认值为5
knn.fit(norm_train_df[cols], norm_train_df['price'])# 进行模型训练
two_features_predictions = knn.predict(norm_test_df[cols])# 预测

In [44]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_features_rmse = two_features_mse ** (1/2)
print(two_features_rmse)

0.824222726134


In [45]:
# 加入更多的特征
cols = ['accommodates', 'bedrooms', 'bathrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews']
knn.fit(norm_train_df[cols], norm_train_df['price'])
four_features_predictions = knn.predict(norm_test_df[cols])
four_features_mse = mean_squared_error(norm_test_df['price'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
print(four_features_rmse)

0.801731509444
